<img align="left" src="https://ithaka-labs.s3.amazonaws.com/static-files/images/tdm/tdmdocs/CC_BY.png"><br />

Created by [Nathan Kelber](http://nkelber.com) and Ted Lawless for [JSTOR Labs](https://labs.jstor.org/) under [Creative Commons CC BY License](https://creativecommons.org/licenses/by/4.0/)<br />
For questions/comments/improvements, email nathan.kelber@ithaka.org.<br />
___

# Latent Dirichlet Allocation (LDA) Topic Modeling

**Description:**
This [notebook](https://docs.constellate.org/key-terms/#jupyter-notebook) demonstrates how to do topic modeling. The following processes are described:

* Using the `constellate` client to retrieve a dataset
* Filtering based on a pre-processed ID list
* Filtering based on a [stop words list](https://docs.constellate.org/key-terms/#stop-words)
* Cleaning the tokens in the dataset
* Creating a [gensim dictionary](https://docs.constellate.org/key-terms/#gensim-dictionary)
* Creating a [gensim](https://docs.constellate.org/key-terms/#gensim) [bag of words](https://docs.constellate.org/key-terms/#bag-of-words) [corpus](https://docs.constellate.org/key-terms/#corpus)
* Computing a topic list using [gensim](https://docs.constellate.org/key-terms/#gensim)
* Visualizing the topic list with `pyldavis`

**Use Case:** For Researchers (Mostly code without explanation, not ideal for learners)

**Difficulty:** Intermediate

**Completion time:** 60 minutes

**Knowledge Required:** 
* Python Basics Series ([Start Python Basics I](./python-basics-1.ipynb))

**Knowledge Recommended:**
* [Exploring Metadata](./metadata.ipynb)
* [Working with Dataset Files](./working-with-dataset-files.ipynb)
* [Pandas I](./pandas-1.ipynb)
* [Creating a Stopwords List](./creating-stopwords-list.ipynb)
* A familiarity with [gensim](https://docs.constellate.org/key-terms/#gensim) is helpful but not required.

**Data Format:** [JSON Lines (.jsonl)](https://docs.constellate.org/key-terms/#jsonl)

**Libraries Used:**
* [constellate](https://docs.constellate.org/key-terms/#tdm-client) client to collect, unzip, and read our dataset
* [pandas](https://constellate.org/docs/key-terms/#pandas) to load a preprocessing list
* `csv` to load a custom stopwords list
* [gensim](https://docs.constellate.org/key-terms/#gensim) to accomplish the topic modeling
* [NLTK](https://docs.constellate.org/key-terms/#nltk) to create a stopwords list (if no list is supplied)
* `pyldavis` to visualize our topic model

**Research Pipeline**
1. Build a dataset
2. Create a "Pre-Processing CSV" with [Exploring Metadata](./exploring-metadata.ipynb) (Optional)
3. Create a "Custom Stopwords List" with [Creating a Stopwords List](./creating-stopwords-list.ipynb) (Optional)
4. Complete the Topic Modeling analysis with this notebook
____

## What is Topic Modeling?

**Topic modeling** is a **machine learning** technique that attempts to discover groupings of words (called topics) that commonly occur together in a body of texts. The body of texts could be anything from journal articles to newspaper articles to tweets.

**Topic modeling** is an unsupervised, clustering technique for text. We give the machine a series of texts that it then attempts to cluster the texts into a given number of topics. There is also a *supervised*, clustering technique called **Topic Classification**, where we supply the machine with examples of pre-labeled topics and then see if the machine can identify them given the examples.

**Topic modeling** is usually considered an exploratory technique; it helps us discover new patterns within a set of texts. **Topic Classification**, using labeled data, is intended to be a predictive technique; we want it to find more things like the examples we give it.

<font color='red'>Read more</font>

* ["Latent Dirichlet Allocation: Intuition, math, implementation and visualisation with pyLDAvis" Ioana](https://towardsdatascience.com/latent-dirichlet-allocation-intuition-math-implementation-and-visualisation-63ccb616e094) 2020
* ["Latent Dirichlet Allocation" Blei, Ng, Jordan](https://www.jmlr.org/papers/volume3/blei03a/blei03a.pdf?TB_iframe=true&width=370.8&height=658.8) 2003

## Import your dataset

We'll use the `constellate` client library to automatically retrieve the dataset in the JSON file format. 

Enter a [dataset ID](https://docs.constellate.org/key-terms/#dataset-ID) in the next code cell.

If you don't have a dataset ID, you can:
* Use the sample dataset ID already in the code cell
* [Create a new dataset](https://constellate.org/builder)
* [Use a dataset ID from other pre-built sample datasets](https://constellate.org/dataset/dashboard)

In [1]:
# Creating a variable `dataset_id` to hold our dataset ID
# The default dataset is Independent Voices
# Independent Voices is an open access digital collection of alternative press newspapers, magazines and journals,
# drawn from the special collections of participating libraries. These periodicals were produced by feminists, 
# dissident GIs, campus radicals, Native Americans, anti-war activists, Black Power advocates, Hispanics, 
# LGBT activists, the extreme right-wing press and alternative literary magazines 
# during the latter half of the 20th century.

dataset_id = "02b8c5c7-64bd-efe3-01d8-88c9efe7d17c"

Next, import the `constellate` client, passing the `dataset_id` as an argument using the `get_dataset` method.

In [2]:
%%time
# Importing your dataset with a dataset ID
import constellate
# Pull in the sampled dataset (1500 documents) that matches `dataset_id`
# in the form of a gzipped JSON lines file.
# The .get_dataset() method downloads the gzipped JSONL file
# to the /data folder and returns a string for the file name and location
#dataset_file = constellate.get_dataset(dataset_id)

# To download the full dataset (up to a limit of 25,000 documents),
# request it first in the builder environment. See the Constellate Client
# documentation at: https://constellate.org/docs/constellate-client
# Then use the `constellate.download` method show below.
dataset_file = constellate.download(dataset_id, 'jsonl')

Constellate: use and download of datasets is covered by the Terms & Conditions of Use: https://constellate.org/terms-and-conditions/
All documents from JSTOR published in American Journal of Archaeology from 1897 - 2020. 12301 documents.
INFO:root:Downloading 02b8c5c7-64bd-efe3-01d8-88c9efe7d17c metadata to /root/data/02b8c5c7-64bd-efe3-01d8-88c9efe7d17c.jsonl.gz


CPU times: user 1.06 s, sys: 664 ms, total: 1.72 s
Wall time: 14.6 s


## Apply Pre-Processing Filters (if available)
If you completed pre-processing with the "Exploring Metadata and Pre-processing" notebook, you can use your CSV file of dataset IDs to automatically filter the dataset. Your pre-processed CSV file  must be in the root folder.

In [5]:
# Import a pre-processed CSV file of filtered dataset IDs.
# If you do not have a pre-processed CSV file, the analysis
# will run on the full dataset and may take longer to complete.
import pandas as pd
import os

pre_processed_file_name = f'data/pre-processed_{dataset_id}.csv'

if os.path.exists(pre_processed_file_name):
    df = pd.read_csv(pre_processed_file_name)
    filtered_id_list = df["id"].tolist()
    use_filtered_list = True
    print('Pre-Processed CSV found. Successfully read in ' + str(len(df)) + ' documents.')
else: 
    use_filtered_list = False
    print('No pre-processed CSV file found. Full dataset will be used.')

No pre-processed CSV file found. Full dataset will be used.


## Load Stopwords List

If you have created a stopword list in the stopwords notebook, we will import it here. (You can always modify the CSV file to add or subtract words then reload the list.) Otherwise, we'll load the NLTK [stopwords](https://docs.constellate.org/key-terms/#stop-words) list automatically.

In [6]:
# Load a custom data/stop_words.csv if available
# Otherwise, load the nltk stopwords list in English

# The filename of the custom data/stop_words.csv file
stopwords_list_filename = 'data/stop_words.csv'

if os.path.exists(stopwords_list_filename):
    import csv
    with open(stopwords_list_filename, 'r') as f:
        stop_words = list(csv.reader(f))[0]
    print('Custom stopwords list loaded from CSV')
else:
    # Load the NLTK stopwords list
    from nltk.corpus import stopwords
    stop_words = stopwords.words('english')
    print('NLTK stopwords list loaded')

NLTK stopwords list loaded


In [7]:
list(stop_words)

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

## Define a Function to Process Tokens
Next, we create a short function to clean up our tokens.

In [8]:
def process_token(token):
    token = token.lower()
    if token in stop_words:
        return
    if len(token) < 4:
        return
    if not(token.isalpha()):
        return
    return token

In [9]:
%%time
# Limit to n documents. Set to None to use all documents.

limit = 3000

n = 0
documents = []
for document in constellate.dataset_reader(dataset_file):
    processed_document = []
    document_id = document["id"]
    if use_filtered_list is True:
        # Skip documents not in our filtered_id_list
        if document_id not in filtered_id_list:
            continue
    unigrams = document.get("unigramCount", {})
    for gram, count in unigrams.items():
        clean_gram = process_token(gram)
        if clean_gram is None:
            continue
        processed_document += [clean_gram] * count # Add the unigram as many times as it was counted
    if len(processed_document) > 0:
        documents.append(processed_document)
    if n % 1000 == 0:
        print(f'Unigrams collected for {n} documents...')
    n += 1
    if (limit is not None) and (n >= limit):
       break
print(f'All unigrams collected for {n} documents.')

Unigrams collected for 0 documents...
Unigrams collected for 1000 documents...
Unigrams collected for 2000 documents...
All unigrams collected for 3000 documents.
CPU times: user 22.7 s, sys: 380 ms, total: 23 s
Wall time: 23 s


Build a gensim dictionary corpus and then train the model. More information about parameters can be found at the [Gensim LDA Model page](https://radimrehurek.com/gensim/models/ldamodel.html).

In [10]:
import gensim
dictionary = gensim.corpora.Dictionary(documents)

INFO:gensim.corpora.dictionary:adding document #0 to Dictionary(0 unique tokens: [])
INFO:gensim.corpora.dictionary:built Dictionary(149869 unique tokens: ['added', 'also', 'amazing', 'amply', 'ancient']...) from 3000 documents (total 4120252 corpus positions)


In [11]:
doc_count = len(documents)
num_topics = 7 # Change the number of topics
passes = 5 # The number of passes used to train the model
# Remove terms that appear in less than 50 documents and terms that occur in more than 90% of documents.
dictionary.filter_extremes(no_below=50, no_above=0.90)

INFO:gensim.corpora.dictionary:discarding 143096 tokens: [('amazing', 34), ('archival', 38), ('avitsur', 2), ('aviv', 32), ('ayalon', 2), ('capacities', 24), ('cessing', 5), ('eretz', 6), ('etan', 1), ('foxhall', 8)]...
INFO:gensim.corpora.dictionary:keeping 6773 tokens which were in no less than 50 and no more than 2700 (=90.0%) documents
INFO:gensim.corpora.dictionary:resulting dictionary: Dictionary(6773 unique tokens: ['added', 'also', 'amply', 'ancient', 'archaeological']...)


In [12]:
bow_corpus = [dictionary.doc2bow(doc) for doc in documents]

In [13]:
bow_corpus[0]

[(0, 1),
 (1, 2),
 (2, 1),
 (3, 1),
 (4, 2),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1),
 (10, 4),
 (11, 1),
 (12, 1),
 (13, 1),
 (14, 5),
 (15, 1),
 (16, 2),
 (17, 1),
 (18, 1),
 (19, 3),
 (20, 1),
 (21, 1),
 (22, 1),
 (23, 1),
 (24, 1),
 (25, 1),
 (26, 1),
 (27, 1),
 (28, 1),
 (29, 1),
 (30, 1),
 (31, 1),
 (32, 1),
 (33, 1),
 (34, 1),
 (35, 1),
 (36, 1),
 (37, 1),
 (38, 1),
 (39, 1),
 (40, 1),
 (41, 1),
 (42, 1),
 (43, 2),
 (44, 1),
 (45, 2),
 (46, 1),
 (47, 2),
 (48, 1),
 (49, 1),
 (50, 1),
 (51, 3),
 (52, 1),
 (53, 1),
 (54, 1),
 (55, 1),
 (56, 1),
 (57, 1),
 (58, 1),
 (59, 1),
 (60, 1),
 (61, 1),
 (62, 1),
 (63, 1),
 (64, 4),
 (65, 1),
 (66, 1),
 (67, 1),
 (68, 1),
 (69, 1),
 (70, 1),
 (71, 1),
 (72, 1),
 (73, 1),
 (74, 2),
 (75, 2),
 (76, 1),
 (77, 1),
 (78, 1),
 (79, 2),
 (80, 1),
 (81, 1),
 (82, 2),
 (83, 1),
 (84, 1),
 (85, 3),
 (86, 1),
 (87, 2),
 (88, 1),
 (89, 1),
 (90, 1),
 (91, 1),
 (92, 1),
 (93, 1),
 (94, 1),
 (95, 1),
 (96, 1),
 (97, 1),
 (98, 1),
 (99, 1),
 (100, 1),

In [14]:
%%time
# Train the LDA model
model = gensim.models.LdaModel(
    corpus=bow_corpus,
    id2word=dictionary,
    num_topics=num_topics,
    passes=passes
)

INFO:gensim.models.ldamodel:using symmetric alpha at 0.14285714285714285
INFO:gensim.models.ldamodel:using symmetric eta at 0.14285714285714285
INFO:gensim.models.ldamodel:using serial LDA version on this node
INFO:gensim.models.ldamodel:running online (multi-pass) LDA training, 7 topics, 5 passes over the supplied corpus of 3000 documents, updating model once every 2000 documents, evaluating perplexity every 3000 documents, iterating 50x with a convergence threshold of 0.001000
INFO:gensim.models.ldamodel:PROGRESS: pass 0, at document #2000/3000
INFO:gensim.models.ldamodel:merging changes from 2000 documents into a model of 3000 documents
INFO:gensim.models.ldamodel:topic #1 (0.143): 0.007*"found" + 0.005*"roman" + 0.005*"also" + 0.005*"would" + 0.004*"ancient" + 0.004*"first" + 0.004*"century" + 0.004*"early" + 0.004*"archaeological" + 0.004*"greek"
INFO:gensim.models.ldamodel:topic #5 (0.143): 0.006*"also" + 0.005*"would" + 0.005*"first" + 0.004*"found" + 0.004*"greek" + 0.004*"univ

INFO:gensim.models.ldamodel:topic #1 (0.143): 0.008*"found" + 0.005*"century" + 0.005*"also" + 0.005*"first" + 0.004*"greek" + 0.004*"museum" + 0.004*"roman" + 0.004*"vases" + 0.004*"made" + 0.004*"three"
INFO:gensim.models.ldamodel:topic #4 (0.143): 0.010*"found" + 0.007*"also" + 0.005*"early" + 0.005*"pottery" + 0.005*"area" + 0.005*"small" + 0.004*"late" + 0.004*"large" + 0.004*"first" + 0.004*"level"
INFO:gensim.models.ldamodel:topic diff=0.171483, rho=0.471405
INFO:gensim.models.ldamodel:PROGRESS: pass 3, at document #2000/3000
INFO:gensim.models.ldamodel:merging changes from 2000 documents into a model of 3000 documents
INFO:gensim.models.ldamodel:topic #4 (0.143): 0.010*"found" + 0.007*"also" + 0.005*"early" + 0.005*"pottery" + 0.005*"area" + 0.005*"late" + 0.005*"small" + 0.005*"large" + 0.004*"bronze" + 0.004*"level"
INFO:gensim.models.ldamodel:topic #5 (0.143): 0.008*"would" + 0.007*"also" + 0.005*"first" + 0.005*"roman" + 0.004*"greek" + 0.004*"left" + 0.004*"head" + 0.004*"

## Perplexity

After each pass, the LDA model will output a "perplexity" score that measures the "held out log-likelihood". Perplexity is a measure of how "surpised" the machine is to see certain data. In other words, perplexity measures how successfully a trained topic model predicts new data. The model may be trained many times with different parameters, optimizing for the lowest possible perplexity.

In general, the perplexity score should trend downward as the machine "learns" what to expect from the data. While a low perplexity score may signal the machine has learned the documents' patterns, that does not mean that the topics formed from a model with low perplexity will form the most coherent topics. (See ["Reading Tea Leaves: How Humans Interpret Topic Models" Chang, et al. 2009](https://papers.nips.cc/paper/2009/hash/f92586a25bb3145facd64ab20fd554ff-Abstract.html).)



## Topic Coherence

The failure of perplexity scores to consistently create "good" topics has led to new methods in "topic coherence". Here we demonstrate two of these methods with Gensim but there are additional methods available. Ideally, a researcher would run many topic models, discovering the optimum settings for topic coherence.

Ultimately, however, the best judgment of topic coherence is a disciplinary expert, particularly someone with familiarity with the materials in question.

<font color='red'>Read more</font>

* ["Optimizing Semantic Coherence in Topic Models" Mimno, et al. 2011](http://dirichlet.net/pdf/mimno11optimizing.pdf)
* ["Automatic Evaluation of Topic Coherence" Newman, et al. 2010](https://mimno.infosci.cornell.edu/info6150/readings/N10-1012.pdf))


In [15]:
# Compute the coherence score using UMass
# u_mass is measured from -14 to 14, higher is better
from gensim.models import CoherenceModel

coherence_model_lda = CoherenceModel(
    model=model,
    corpus=bow_corpus,
    dictionary=dictionary, 
    coherence='u_mass'
)

# Compute Coherence Score using UMass
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

INFO:gensim.topic_coherence.text_analysis:CorpusAccumulator accumulated stats from 1000 documents
INFO:gensim.topic_coherence.text_analysis:CorpusAccumulator accumulated stats from 2000 documents
INFO:gensim.topic_coherence.text_analysis:CorpusAccumulator accumulated stats from 3000 documents

Coherence Score:  -0.5817490844361205


## Display a List of Topics
Print the most significant terms, as determined by the model, for each topic.

In [16]:
for topic_num in range(0, num_topics):
    word_ids = model.get_topic_terms(topic_num)
    words = []
    for wid, weight in word_ids:
        word = dictionary.id2token[wid]
        words.append(word)
    print("Topic {}".format(str(topic_num).ljust(5)), " ".join(words))

Topic 0     professor wall school building west temple north south american east
Topic 1     found also century vases first museum greek made three figure
Topic 2     roman greek temple also inscriptions century early american found museum
Topic 3     also early evidence late roman bronze archaeological greek first edited
Topic 4     found also early area pottery small large late level bronze
Topic 5     would also first roman left greek must right head could
Topic 6     university archaeological american archaeology institute journal book volume ancient classical


## Visualize the Topic Distances

Visualize the model using [`pyLDAvis`](https://pyldavis.readthedocs.io/en/latest/). This visualization can take a while to generate depending on the size of your dataset.

Try choosing a topic and adjusting the λ slider. When λ approaches 0, the words in a given document occur almost entirely in that topic. When λ approaches 1, the words occur more often in other topics.

In [17]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(model, bow_corpus, dictionary)

/usr/local/lib/python3.8/site-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.022036 -0.116415       1        1  21.262081
5     -0.073039  0.012621       2        1  19.718575
4     -0.097574 -0.057495       3        1  18.427160
1     -0.057679  0.042156       4        1  17.625152
0      0.014039  0.042020       5        1   9.060255
6      0.168008 -0.014940       6        1   7.436101
2      0.024210  0.092054       7        1   6.470676, topic_info=                Term          Freq         Total Category  logprob  loglift
111       university  11409.000000  11409.000000  Default  30.0000  30.0000
1053       professor   5619.000000   5619.000000  Default  29.0000  29.0000
501         american   8534.000000   8534.000000  Default  28.0000  28.0000
4     archaeological  10683.000000  10683.000000  Default  27.0000  27.0000
510      archaeology   8884.000000   8884.000000  Default  26.0000  26.0000
...              ...           ...           ...      ...      ...      ...
530           bronze    720.378277   9398.571768   Topic7  -5.6960   0.1694
111       university    716.941592  11409.023290   Topic7  -5.7008  -0.0293
33             first    731.266337  13594.785344   Topic7  -5.6810  -0.1848
1003            late    676.001590  10248.204348   Topic7  -5.7596   0.0192
676             near    612.672962   6371.332612   Topic7  -5.8579   0.3962

[600 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
5272      1  0.071233  abstracts
5272      4  0.019427  abstracts
5272      5  0.071233  abstracts
5272      6  0.828896  abstracts
5272      7  0.006476  abstracts
...     ...       ...        ...
829       3  0.024744       zeus
829       4  0.113117       zeus
829       5  0.055380       zeus
829       6  0.008248       zeus
829       7  0.126078       zeus

[3030 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 6, 5, 2, 1, 7, 3])

In [18]:
# Export this visualization as an HTML file
# An internet connection is still required to view the HTML
p = pyLDAvis.gensim.prepare(model, bow_corpus, dictionary)
pyLDAvis.save_html(p, 'my_visualization.html')

/usr/local/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.8/site-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
